# Binary_Classification(table_data)

# Setting

In [ ]:
# 1) matplotlib 설정
plt.rc("font", family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
plt.rc('figure', figsize=(8, 4))
plt.rc('figure', dpi=100)

# 2) 이렇게 seaborn scheme을 세팅해도 됨
plt.style.use('seaborn')
sns.set(font_scale=1.5)

import missningno as msno

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 해야할 프로세스
1. 데이터셋 확인 - null data 확인, 수정
2. 탐색적 데이터 분석(Exploratory data analysis - 여러 feature 개별분석, 상관관계 확인, 시각화 툴로 insight
3. feature engineering - OneHot Encoding, Class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등
4. model
5. fit & predict
6. test - 풀려는 문제에 따라 모델 평가방식이 달라짐

# 데이터 불러오기

In [ ]:
DEBUG = False

In [ ]:
if DEBUG:
    NROWS = 50000
else:
    NROWS = None

In [ ]:
# 1) 데이터가 csv인 경우
df_train = pd.read_csv('', nrows=NROWS)
df_test = pd.read_csv('', nrows=NROWS)

# 또는
import os
path = '경로/'
df_train = pd.read_csv(os.path.join(path, '파일명'), nrows=NROWS)
df_test = pd.read_csv(os.path.join(path, '파일명'), nrows=NROWS)

In [ ]:
# 데이터가 너무 많으면 샘플링해서 EDA하기
데이터 = 데이터.sample(frac=0.1)

# imbalanced data가 걱정이면
from sklearn.model_selection import StratifiedKFold

fold = StratifiedKFold(n_splits=10)
for trn_idx, val_idx in fold.split(데이터, 데이터['타겟']):
    break
데이터 = 데이터.iloc[trn_idx]

In [ ]:
# 비트 수로 데이터 용량 줄이기
for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)    

# 1. Dataset 확인

In [ ]:
# 훈련, 테스트 데이터의 칼럼이 같은지 확인
set(trainset.columns) - set(testset.columns)

In [ ]:
데이터.head()

In [ ]:
데이터.describe() # describe()는 숫자형 데이터만 보여준다

# 특정 열을 지정해서 describe() 하면 count와 unique값을 계산해 줌

- null data가 존재하는 열(feature)을 확인
- 변수의 자료형 확인

In [ ]:
데이터.info() # info()는 모든 데이터를 보여주고 데이터 타입도 알 수 있다(dtypes로도 확인 가능)

In [ ]:
데이터.drop_duplicates() # 중복된 행이 있으면 안되는 경우 제거

## 1.1 메타 데이터
- **role**: input, ID, target
- **level**: binary, categorical, ordinal, interval
- **keep**: True or False
- **dtype**: int, float, str

**porto 데이터셋에만 해당하는 코드이므로 차후에 일반화된 코드로 바꿀 것**

In [ ]:
data = []
for f in train.columns:
    # Defining the role
    if f == '타겟변수':
        role = 'target'
    elif f == '아이디':
        role = 'id'
    else:
        role = 'input'
        
    # Defining the level
    if 'bin' in f or f == '타겟변수':
        level = 'binary'
    elif 'cat' in f or f == '아이디':
        level = 'categorical'
    elif train[f].dtype == float:
        level = 'interval'
    elif train[f].dtype == 'int64':   # int32, 64 등 확인하고 설정할 것
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == '아이디':
        keep = False
    
    # Defining the data type
    dtype = train[f].dtype
    
    category = 'none'
    # 칼럼 이름에 따른 어떤 분류가 필요한 경우
    if 'ind' in feature:
        category = 'individual'
    elif 'reg' in feature:
        category = 'registration'
    elif 'car' in feature:
        category = 'car'
    elif 'calc' in feature:
        category = 'calculated'
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype,
        'category': category
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'role', 'level', 'keep', 'dtype', 'category'])
meta.set_index('varname', inplace=True)

- meta의 변수 level 잘 할당되었는지 확인

#### 메타데이터 확인 예시

In [ ]:
meta[(meta.level == 'categorical') & (meta.keep)].index

In [ ]:
pd.DataFrame({'count': meta.groupby(['role', 'level'])['role'].size()}).reset_index() # size includes NaN values, count does not

## 1.2 Null data check
- isnull().sum(), msno, bar 등으로 null값 시각화

In [ ]:
# isnull()로 시각화
# 훈련 데이터, 테스트 데이터 확인할 것
vars_with_missing = []

for f in 데이터.columns:
    missings = 데이터[f].isnul().sum()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings / 데이터.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))



# Null값이 NaN이 아닌 다른 값으로 (-1) 표시되어 있을 때
vars_with_missing = []

for f in 데이터.columns:
    missings = 데이터[데이터[f] == -1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings / 데이터.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

- 데이터에 존재하는 null 값이 퍼센트로 표시된다
- 그냥 데이터.isnull().sum()으로 해도됨

#### MSNO matrix로 시각화

In [ ]:
# 훈련 데이터, 테스트 데이터 확인할 것

msno.matrix(df=데이터.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2)) 

# color는 RGB
# 굳이 .iloc 안 붙여도 됨
# 이 차트는 null값의 분포를 보기 좋음

In [ ]:
# msno의 bar로 시각화
# 훈련 데이터, 테스트 데이터 확인할 것

msno.bar(df=데이터.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2))

# 이 차트는 null값의 비율을 보기 좋음

## 1.3 Target label 확인
- target label의 분포를 확인

- binary classification 문제의 경우 1과 0의 분포에 따라 모델 평가 방법이 달라진다.

In [ ]:
# 데이터 타입을 확인(카테고리형, 연속형, 순서형, 이진형, 문자열 등등)
데이터['타겟'].value_counts()  # 칼럼을 두개 지정하면?? 그 데이터로 bar() 하면??

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

# Pie plot
데이터['타겟'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True) # explode 는 범주값 개수만큼 지정
ax[0].set_title('Pie plot - 타겟')
ax[0].set_ylabel('')
# Count plot
sns.countplot('타겟', data=데이터, ax=ax[1])
ax[1].set_title('Count plot - 타겟')

plt.show()

- Target label의 분포가 균일한지 확인해야 한다. 분포가 극단적으로 치우쳐져 있다면 accuracy만으로 성능 측정 불가

### 1.3.1 Target label이 불균형할 때

### 1.3.1.1 undersampling

#### Random undersampling

In [ ]:
from sklearn.utils import shuffle

desired_apriori = 0.10

# Get the indices per target value
idx_0 = 데이터[데이터['타겟'] == 0].index
idx_1 = 데이터[데이터['타겟'] == 1].index

# Get original number of records per target value
nb_0 = len(데이터.loc[idx_0])
nb_1 = len(데이터.loc[idx_1])

# Calculatet the undersampling rate and resulting number of records with target=0
undersampling_rate = ((1 - desired_apriori) * nb_1) / (desired_apriori * nb_0)
undersampled_nb_0 = int(undersampling_rate * nb_0)
print('Rate to undersample records with target=0: {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# Randomly select records with target=0 to get at the desired a priori
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# Construct list with remaining indices
idx_list = list(undersampled_idx) + list(idx_1)

# Return undersample data frame
데이터 = 데이터.loc[idx_list].reset_index(drop=True)

## 1.4 Outlier 확인
- 연속형 feature에서 사용
- null값이 있는 칼럼은 np.percentile로 계산 안되므로 isnull() or notnull()을 이용해 null값 제외하고 계산
- train데이터에서만 수행해야 한다. valid, test에서 하면 안됨

### 1.4.1 아웃라이어를 2칼럼 이상 포함하고 있는 행을 지우는 방법
- IQR로 아웃라이어 탐지

In [ ]:
from collections import Counter

# Outlier detection
# 아웃라이어가 n개 이상 있는 행 조사
def detect_outliers(df, n, features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according to the Tukey method.
    """
    
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[df[col].isnull() == False][col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[df[col].isnull() == False][col], 75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step)].index
        
        # append the found outlier indices for col to the list of outlier indices
        outlier_indices.extend(outlier_list_col)
    
    # select obervations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)           # 시퀀스 안의 각 자료들이 몇개인지 세어주는 함수(딕셔너리 반환)
    multiple_outliers = list(k for k,v in outlier_indices.items() if v > n)
    
    return multiple_outliers

outliers_to_drop = detect_outliers(데이터, n개, ['변수1', '변수2', '변수3'])
데이터.loc[outliers_to_drop]

### 1.4.2 각 feature마다 아웃라이어를 체크하는 방법

In [ ]:
# IQR로 아웃라이어 체크
Q1 = np.percentile(데이터[데이터['변수'].isnull() == False]['변수'], 25)
Q3 = np.percentile(데이터[데이터['변수'].isnull() == False]['변수'], 75)
IQR = Q3 - Q1

outlier_step = 1.5 * IQR
outlier_indices = (데이터['변수'] < Q1 - outlier_step) | (데이터['변수'] > Q3 + outlier_step)

outlier = 데이터[outlier_indices]
non_outlier = 데이터[~outlier_indices]

# 히스토그램 시각화를 통해 체크
sns.distplot(데이터['변수'], bins=, kde=False)

- Home Credit competition `DAYS_EMPLOYED`변수
    - 극단값이 꽤 많아서 히스토그램을 그려보면 마치 정상값, 이상값의 binary 변수처럼 보이는 경우

#### 아웃라이어 데이터와 정상 데이터의 Target 비율 비교

In [ ]:
outlier['타겟'].mean()

non_outlier['타겟'].mean()

- 아웃라이어와 정상 데이터의 Target 비율이 의미있는 차이를 보인다면 삭제하지 않고 이용할 수 있다

#### 아웃라이어 drop

In [ ]:
데이터 = 데이터.drop(outliers_to_drop, axis=0).reset_index(drop=True)

## 1.5 train, test 데이터의 feature 분포 확인
- 각 feature들이 train, test 에서 모두 같은 분포를 이루고 있지 않으면 서로 다른 데이터임

In [ ]:
sns.set_style('whitegrid')

plt.figure()
fig, ax = plt.subplots(행, 열, figsize=(12, 4))
i = 0
for feature in 데이터:
    i = i + 1
    plt.subplot(행, 열, i)
    sns.kdeplot(훈련[feature], label='train')
    sns.kdeplot(테스트[feature], label='test')
    plt.xlabel(feature, fontsize=12)
    plt.ylabel('Distribution', fontsize=12)
    locs, labels = plt.xticks()
#     plt.setp(labels, rotation=90)
    plt.tick_params(axis='both', which='major', labelsize=12)
plt.show()

# 2. Exploratory data analysis
- 데이터 안에 숨겨진 사실을 찾기 위해 적절한 시각화 필요
- 시각화 라이브러리는 matmplotlib, seaborn, plotly 등. 목적에 맞게 소스코드를 정리해 둘 것
- 시각화 할 때 눈금 조정으로 인한 실제 값과 라벨의 미스매치 항상 조심할 것
- 관측값이 아닌 비율값을 시각화 할 때는 항상 신뢰구간이 보이는 차트를 사용할 것.(데이터가 적은 경우 문제가 됨)

## 2.1 일변수 분석
- 변수의 값에 따라서 Target label값이 어떻게 다른지를 groupby 또는 pivot으로 확인

### 2.1.1 카테고리형, 이진형 데이터 (서수형도 포함)

In [ ]:
# 이진 데이터
binary = meta[(meta.level == 'binary') & (meta.keep)].index
데이터[binary].describe()

In [ ]:
# 카테고리 데이터
category = meta[(meta.level == 'categorical') & (meta.keep)].index
데이터[category].describe()

In [ ]:
# 서수형 데이터
ordinal = meta[(meta.level == 'ordinal') & (meta.keep)].index
데이터[ordinal].describe()

#### 모든 변수에 대해서 변수 자체의 분포 확인
범주 값이 많은 카테고리형 변수는 한 plot에서 확인 못함

In [ ]:
# binary 변수
bin_col = [col for col in binary if col != '타겟']
zero_list = []
one_list = []
for col in bin_col:
    zero_list.append((데이터[col]==0).sum() / 데이터.shape[0] * 100)
    one_list.append((데이터[col]==1).sum() / 데이터.shape[0] * 100)

plt.figure()
fig, ax = plt.subplots(figsize=(6, 6))
p1 = sns.barplot(x=bin_col, y=zero_list, color='blue', ax=ax)
p2 = sns.barplot(x=bin_col, y=one_list, bottom=zero_list, color='red', ax=ax)
plt.xlabel('Binary features', fontsize=12)
plt.ylabel('Percent of zero/one [%]', font_size=12)
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.tick_params(axis='both', which='major', labelsize=12)
plt.legend((p1, p2), ('Zero', 'One'))
plt.show()

# ordinal

# categorical 변수


- 만약 어떤 변수가 모두 0 또는 1인 분포라면 의미 없는 변수

#### 모든 변수에 대해서 Target 비율 분석

In [ ]:
# binary 변수
bin_col = [col for col in binary if col != '타겟']
i = 0
t1 = 데이터.loc[데이터['타겟'] != 0]
t0 = 데이터.loc[데이터['타겟'] == 0]

plt.figure()
fig, ax = plt.subplots(6, 3, figsize=(12, 24))

for feature in bin_col:
    i += 1
    plt.subplot(행, 열, i)
    sns.barplot(feature, '타겟', data=데이터, palette='Set2')
#   빠르게 하려면 비율을 직접 구해서 바로 넣을수도 있음
    plt.ylabel('Density plot', fontsize=12)
    plt.xlabel(feature, fontsize=12)
    locs, labels = plt.xticks()
    plt.tick_params(axis='both', which='major', labelsize=12)
plt.show()

# category 변수
category = meta[(meta.level == 'categorical') & (meta.keep)].index

for feature in category:
    fig, ax = plt.subplots(figsize=(6, 6))
    # Calculate the percentage of target=1 per category value
    cat_perc = 데이터[[feature, '타겟']].groupby([feature], as_index=False).mean()
    sns.barplot(x=feature, y='타겟', data=cat_perc, order=cat_perc[feature], ax=ax)
    plt.ylabel('Percent of target with value 1 [%]', fontsize=12)
    plt.xlabel(feature, fontsize=12)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.show()

- 시각화 했을 때 신뢰구간이 너무 크면 데이터가 부족하므로 해당 데이터의 신뢰성에 문제가 있을 수 있음
    - 비율이 0이거나 1인 데이터 조심(신뢰구간 표시 안됨)

#### 모든 변수에 대해서 target 분포 확인

In [ ]:
# binary 변수
bin_col = [col for col in binary if col != '타겟']
i = 0
t1 = 데이터.loc[데이터['타겟'] != 0]
t2 = 데이터.loc[데이터['타겟'] == 0]

sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(행, 열, figsize=(6, 6))

for feature in bin_col:
    i += 1
    plt.subplot(행, 열, i)
    sns.kdeplot(t1[feature], label='target = 1')
    sns.kdeplot(t0[feature], label='target = 0')
    plt.xlabel(feature, fontsize=12)
    plt.ylabel('Density plot', fontsize=12)
    locs, labels = plt.xticks()
    plt.tick_params(axis='both', which='major', labelsize=12)
plt.show()

# category 변수
category = meta[(meta.level == 'categorical') & (meta.keep)].index
i = 0
t1 = 데이터.loc[데이터['타겟'] != 0]
t0 = 데이터.loc[데이터['타겟'] == 0]

sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(행, 열, figsize=16, 16)
for feature in category:
    i += 1
    plt.subplot(행, 열, i)
    sns.kdeplot(t1[feature], label='target = 1')
    sns.kdeplot(t0[feature], label='target = 0')
    plt.xlabel(feature, fontsize=12)
    plt.ylabel('Density plot', fontsize=12)
    locs, labels = plt.xticks()
    plt.tick_params(axis='both', which='major', labelsize=12)
plt.show()

#### 변수 하나씩 process

#### 변수의 각 범주마다 target 데이터의 전체 관찰값의 균일한지 확인

In [ ]:
데이터[['변수', '타겟']].groupby(['변수'], as_index=True).count()  # 총 데이터 개수. value_counts()와 같음

#### target 데이터 범주도 균일한지 확인(타겟이 카테고리인 경우도 활용 가능)

In [ ]:
pd.crosstab(데이터['변수'], 데이터['타겟'], margins=True).style.background_gradient(cmap='summer_r')
# 색상 google에 color scheme 검색

- 변수 값에 따라 Target값이 어떻게 다른지 확인하고 기록

#### 변수의 분포 
#### 변수의 각 범주에 따른 target값이 1인 비율 시각화
#### countplot으로 변수에 따른 Target 쪼개서 시각화

In [ ]:
y_position = 1.02
f, ax = plt.subplots(1, 3, figsize=(20, 8))

# 변수의 분포
sns.countplot('변수', data=데이터, ax=ax[0])  # countplot은 bar()와 달리 색깔을 다 다르게 입혀줌
# 데이터['변수'].value_counts().plot.bar(color=['#CD7F32', '#FFDF00', '#D3D3D3'], ax=ax[0]) x축이 값이 옆으로 출력됨
ax[0].set_title('Number of 타겟 By 변수', y=y_position)
ax[0].set_ylabel('Count')

# 변수에 따른 Target label이 1인 값 비율 시각화
sns.barplot('변수', '타겟', data=데이터, palette='Set2', order=, ax=ax[1]) # 색깔이 다 다름, barplot도 비율을 계산
# 데이터[['변수', '타겟']].groupby(['변수'], as_index=True).mean().plot.bar(ax=ax[1]) 로 시각화하면 신뢰구간을 볼 수 없다!!
# factorplot으로도 가능
ax[1].set_title('타겟 ratio by 변수')

# Target 분포 시각화(Target 쪼갬)
# t1 = 데이터.loc[데이터['타겟'] != 0]
# t0 = 데이터.loc[데이터['타겟'] == 0]
# sns.kdeplot(t1['변수'], ax=ax[2])
# sns.kdeplot(t0['변수'], ax=ax[2])
sns.countplot('변수', hue='타겟', data=데이터, ax=ax[2])
# countplot에서 hue에 어떤 칼럼을 지정하면 그 칼럼의 범주에 따라 bar 차트를 쪼갠다 

ax[2].set_title('변수: a vs b', y=y_position)

plt.show()

# xlabel 순서 달라져서 각 plot의 x축 순서가 바뀌면 sort_values(ascending=)또는 sort_index()으로 바꿔주기(혹은 order로 순서 바꿀 수 있음)
# 눈금 바꿀 때 실제 값과 미스매치 날 수 있으니 항상 조심!!!

- 시각화 했을 때 신뢰구간이 너무 크면 데이터가 부족하므로 해당 데이터의 신뢰성에 문제가 있을 수 있음
    - 비율이 0이거나 1인 데이터 조심(신뢰구간 표시 안됨)
    - 해당 데이터를 사용하지 않는 방법도 고려해야 한다
    - barplot에 신뢰구간을 표시하려면 ('칼럼', data=dataframe) 형식으로 전달해야 함.  (x, y) 식으로 넣으면 신뢰구간을 구할 수 없다
    
- category 데이터에서는 이러한 문제를 mean encoding이나 category encoding 등으로 해결한다

In [ ]:
# 구체적인 수치(비율)
데이터[['변수', '타겟']].groupby(['변수'], as_index=True).mean()

# 타겟이 1인 값들의 총합(타겟 레이블이 0, 1 인 경우만 사용)
데이터[['변수', '타겟']].groupby(['변수'], as_index=True).sum()    

- 구체적인 수치로 보고 이 변수가 Target을 예측하는데 도움이 되는지 안되는지 판단

#### 카테고리 혹은 서수형 데이터를 이진화할 필요가 있는 경우
- ex) Dietanic : FamilySize가 1 이냐 or 2 이상이냐 로 이진화해서 새로운 시각으로 볼 수 있다
- 새로운 Feature를 생성

In [ ]:
# 기존 변수에서 특정 범주에 해당하느냐 안하느냐로 이진화한 새 변수 생성
데이터['새 변수'] = 0
데이터.loc[데이터['기존 변수'] == '이진화 범주', '새 변수'] = 1

### 2.1.2 연속형 실수 데이터

#### 모든 연속형 실수 데이터를 시각화

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
train[v].describe()

In [ ]:
i = 0
t1 = 데이터.loc[데이터['타겟'] != 0]
t0 = 데이터.loc[데이터['타겟'] == 0]

sns.set_style('whitegrid')
plt.figure()
fig, ax = plt.subplots(행, 열, figsize=(16, 12))

for feature in v:
    i += 1
    plt.subplot(행, 열, i)
    sns.kdeplot(t1[feature], bw=, label='target = 1')
    sns.kdeplot(t0[feature], bw=, label='target = 0')
    plt.xlabel(feature, fontsize=12)
    plt.ylabel('Density plot', fontsize=12)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.show()

- 한꺼번에 시각화 해서 그 중 target값이 0이냐 1이냐에 따라 다른 분포를 보이는 변수들을 찾아서 시각화 하는게 효율적
- 혹은 target과의 corr만 확인해서 그중 관계성이 높은 것만 탐색하는 방법도 있음

#### 기술 통계 확인

In [ ]:
데이터['변수'].max()
데이터['변수'].min()
데이터['변수'].mean()
데이터['변수'].median()

# 이걸로 한방에
데이터['변수'].describe()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 8))

# 변수 자체의 분포
sns.distplot(데이터['변수'], color='b', bins=, kde=, label='Skewness : {:.2f}'.format(데이터['변수'].skew()), ax=ax[0])
ax[0].legend(loc='best')

# 연속형 변수를 구간화(binning) 해서 Target의 비율을 확인(시각화)
bin_interval = 간격
range_target_ratio = []
sem_by_range = []     # sem : standard error of the mean : 표준오차
for i in range(변수최솟값, 변수최댓값, bin_interval):
    # 구간별 타겟의 비율 p
    p_by_range = 데이터[(데이터['변수'] >= i) & (데이터['변수'] < i + bin_interval)]['타겟'].mean() # 합산값을 구하고 싶다면 sum()
    range_target_ratio.append(p_by_range)
    # 표준오차 구하기
    n_by_range = 데이터[(데이터['변수'] >= i) & (데이터['변수'] < i + bin_interval)]['타겟'].count()
    sem_by_range.append(np.sqrt(p_by_range * (1 - p_by_range)) / np.sqrt(n_by_range))

sns.barplot(np.arange(변수최솟값, 변수최댓값, bin_interval), range_target_ratio, ax=ax[1], yerr=np.array(sem_by_range)*1.96) # 95%신뢰구간
ax[1].set_title('타겟 rate change depending on rage of 변수', y=1.02)
ax[1].set_ylabel('타겟 rate')
ax[1].set_xlabel('Range of 변수(0~x)')
ax[1].set_xticks(np.arange(bin_interval))

plt.show()


# 연속형 변수를 구간화(binning) 해서 Target의 비율을 확인(단순 수치 확인)
변수_data = 데이터[['타겟', '변수']]
변수_data['변수_BINNED'] = pd.cut(데이터['변수'], bins=np.linspace(최소, 최대, 분할))
변수_groups = 변수_data.groupby('변수_BINNED').mean()

#### 변수 자체의 분포, 변수 구간화한 후 Target의 비율

- 변수 자체의 분포 에서
    - 변수가 정규분포인지 균일분포인지, 불균형 데이터는 아닌지 파악
    - null값 처리를 아직 하지 않았으므로 불균형 분포라고 지금 log변환하면 안된다!!!!!!
        - feature engineering에서 null처리, log변환 후 다시 돌아와서 EDA
        - null값이 없으면 log취해도 됨
    - 연속형 변수의 분포를 확인할 때 주의점
        - 이상치 값으로 인해 시각화에 왜곡이 생길경우 이상치를 제외한 분포도 확인해 바야 함(Home Credit Comepetition의 DAYS_EMPLOYED 변수)
    
- 연속형 변수를 구간화(binning) 후 Target의 비율에서
    - 구간은 나누는 방법이 매우 중요
    - 이렇게 시각화 하면 신뢰구간 안 나옴
    - 신뢰구간이 나오지 않으므로 왼쪽 플롯의 실제 관측값의 개수와 비교해 볼 것
    - 신뢰구간 표시하려면 표준편차 구해서 xerr혹은yerr 인자로 전달(0 이나 1같은 극단적인 값은 신뢰구간 못구하니까 조심)

#### 변수값에 따른 Target의 kde(커널밀도추정) 또는 히스토그램

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 5))
sns.kdeplot(데이터[데이터['타겟'] == 0]['변수'], ax=ax)
sns.kdeplot(데이터[데이터['타겟'] == 1]['변수'], ax=ax)
plt.legend(['타겟 == 0', '타겟 == 1'])
plt.show()

# 데이터[데이터['타겟'] == 1]['변수'].plot(kind='kde')
# 데이터[데이터['타겟'] == 0]['변수'].plot(kind='kde') 로도 표현가능

# 정확한 값을 보고싶다면 히스토그램을 그려야 하는데
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

sns.distplot(데이터[데이터['변수'] == 1]['타겟'], bins=, kde=, ax=ax[0]) # 이렇게 하면 kde까지 볼 수 있음
# 데이터[데이터['타겟'] == 1]['변수'].plot.hist(ax=ax[0], color='b', bins=, edgecolor='k')  # plot.hist() : plot(kind='hist')
ax[0].set_title('타겟 == 1')
ax[0].set_xlim([ , ])
ax[0].set_xticks(np.arange( , , ))

sns.distplot(데이터[데이터['변수'] == 0]['타겟'], bins=, kde=, ax=ax[0])
# 데이터[데이터['타겟'] == 0]['변수'].plot.hist(ax=ax[1], color='b', bins=, edgecolor='k')
ax[1].set_title('타겟 == 0')
ax[1].set_xlim([ , ])
ax[1].set_xticks(np.arange( , , ))

plt.show()

- kde의 경우 비율 값 아님, 관측값 기준
- Target값이 0일 때의 변수의 그래프, Target값이 1일 때의 변수의 그래프를 비교했을 때 차이를 보면 
- 연속형 변수의 값에 따른 Target label의 분포를 알 수 있다
- 연속형 변수의 분포를 확인할 때 주의점
    - 이상치 값으로 인해 시각화에 왜곡이 생길경우 이상치를 제외한 분포도 확인해 바야 함(Home Credit Comepetition의 DAYS_EMPLOYED 변수)

#### 변수2(연속형)를 구간화 하지 않고 누적시켜 가면서 타겟의 변화를 시각화 하려면

In [ ]:
cummulate_변수_ratio = []

for i in range(변수최솟값 , 변수최댓값):
    cummulate_변수_ratio.append(데이터[데이터['변수'] < i]['타겟'].sum() / len(데이터[데이터['변수'] < i]['타겟']))
    
plt.figure(figsize=(7, 7))
plt.plot(cummulate_변수_ratio)
plt.title('타겟 rate change depending on range of 변수')
plt.ylabel('타겟 rate')
plt.xlabel('Range of 변수(0~x)')
plt.show()

## 2.2 이변수 분석
- 한 변수에 따라 다른 한 변수의 분포 확인

### 2.2.1 두 변수 모두 카테고리형 변수

In [ ]:
pd.crosstab(데이터['변수1'], 데이터['변수2'], margins=True).style.background_gradient(cmap='summer_r')

- 두 변수 모두 순서형 데이터인 경우에는 상관계수로 heatmap을 그려볼 수 있음

#### 두 변수의 분포를 관찰횟수로 시각화

In [ ]:
plt.figure(figsize=(16, 8))
sns.countplot('변수1', hue='변수2', data=데이터)

# sns.factorplot('변수1', col='변수2', kind='count', data=데이터) 로도 가능

#### 두 변수의 분포를 비율로 시각화 - pie plot
- 변수 1의 각 범주에서 변수2의 분포가 어떻게 되는지를 시각화

In [ ]:
# Pie plot
f, ax = plt.subplots(1, 3, figsize=(20, 8))

train[train['변수1'] == '범주1']['변수2'].value_counts().sort_index().plot.pie(explode=[0, 0, 0], autopct='%1.1f%%', shadow=True, ax=ax[0])
ax[0].set_title('변수2 in 범주1')
ax[0].legend(loc='upper right')

train[train['변수1'] == '범주2']['변수2'].value_counts().sort_index().plot.pie(explode=[0, 0, 0], autopct='%1.1f%%', shadow=True, ax=ax[1])
ax[1].set_title('변수2 in 범주2')
ax[1].legend(loc='upper right')

train[train['변수1'] == '범주3']['변수2'].value_counts().sort_index().plot.pie(explode=[0, 0, 0], autopct='%1.1f%%', shadow=True, ax=ax[2])
ax[2].set_title('변수2 in 범주2')
ax[2].legend(loc='upper right')

plt.show()

# 라벨 색상이 모두 같게 해야 하는데 실수의 여지가 있으므로 반드시 세 차트 모두 라벨 표시

- 변수1과 타겟에 대한 일변수 분석을 할때 의문이 이변수 분석에서 나올 수 있다.
- Titanic의 EDA to Dietanic Notebook에서 SibSp가 높을수록 생존률이 높다가 3명 이상부터 떨어지는데 그 원인이 SibSp가 3 이상인 가족은 모두 Pclass가 3이었음

### 2.2.2 변수1: 카테고리, 변수2: 연속형

#### kdeplot(한 figure에 모두 표시)
#### kdeplot은 비율로 표시됨

In [ ]:
plt.figure(figsize=(8, 6))

데이터['변수2(연속형)'][데이터['변수1(카테고리형)'] == 범주1].plot(kind='kde') 
데이터['변수2(연속형)'][데이터['변수1(카테고리형)'] == 범주2].plot(kind='kde')
데이터['변수2(연속형)'][데이터['변수1(카테고리형)'] == 범주3].plot(kind='kde')

plt.xlabel('변수2(연속형)')
plt.title('변수2 Distribution within classes')
plt.legend(['1st Class', '2nd Class', '3rd Class', ...])

- 이 방식이 violinplot보다 가독성이 좋다

#### 각각 나눠서 표시
#### histogram은 관측값으로 표시됨

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(20, 6))

데이터['변수2(연속형)'][데이터['변수1(카테고리형)'] == 범주1].plot(kind='hist', ax=ax[0], xlim=[ , ]) # x축 스케일 통일
데이터['변수2(연속형)'][데이터['변수1(카테고리형)'] == 범주2].plot(kind='hist', ax=ax[1], xlim=[ , ])
데이터['변수2(연속형)'][데이터['변수1(카테고리형)'] == 범주3].plot(kind='hist', ax=ax[2], xlim=[ , ])
...
plt.xlabel('변수2(연속형)')
plt.title('변수2 Distribution within classes')
plt.legend(['1st Class', '2nd Class', '3rd Class'])

#### distplot은 histogram을 비율로 표시

In [ ]:
f, ax = plt.subplots(1, 3, figsize=(20, 6))

sns.distplot(데이터[데이터['변수1(카테고리형)'] == 범주1]['변수2(연속형)'], ax=ax[0])
ax[0].set_title('변수2 in 변수1의 범주1')
ax[0].set_xlim([ , ])

sns.distplot(데이터[데이터['변수1(카테고리형)'] == 범주2]['변수2(연속형)'], ax=ax[1])
ax[1].set_title('변수2 in 변수1의 범주2')
ax[1].set_xlim([ , ])

sns.distplot(데이터[데이터['변수1(카테고리형)'] == 범주3]['변수2(연속형)'], ax=ax[2])
ax[2].set_title('변수2 in 변수1의 범주3')
ax[2].set_xlim([ , ])

# x축 스케일 통일해야 왜곡 없다

- 변수1의 각 범주에 따라 변수2의 분포를 살펴본다

### 2.2.3 두 변수 모두 연속형

#### 모든 연속형 변수에 대해 상관계수 

In [ ]:
def corr_heatmap(v):
    correlations = 데이터[v].corr()
    
    # Create color map ranging between two colors
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(correlations, cmap=cmap, vmax=1.0, center=0, fmt='.2f', square=True, linewidths=0.5, annot=True,
                cbar_kws={'shrinks': 0.75})
    plt.show()

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index
corr_heatmap(v)

- heatmap상 correlation이 나타나지 않는다고 두 변수가 상관성이 없다고 단정할 수 없음
- 두 변수 모두 순서형 데이터인 경우에도 그려볼 수 있음

#### 상호정보량

### 2.2.4 변수1에 대해 다른 모든 변수로 이변수 분석
- 일변수 분석으로 하나씩 늘려갈 때 마다 이전 분석한 일변수들과 이변수 분석

In [ ]:
f, ax = plt.subplots(다른 모든 변수 갯수만큼, figsize=(15, 10))

# 변수1(카테고리형) 분포
sns.countplot('변수1', data=데이터, ax=ax[0, 0])
ax[0, 0].set_title('(1) No. Of 변수1')

# 변수2(카테고리형)로 split
sns.countplot('변수1', hue='변수2', data=데이터, ax=ax[ , ])
ax[ , ].set_title('(2) 변수2 split for 변수1')

# 변수3(연속형)으로 split
데이터['변수3(연속형)'][데이터['변수1(카테고리형)'] == 범주1].plot(kind='kde', ax=ax[ , ])
데이터['변수3(연속형)'][데이터['변수1(카테고리형)'] == 범주2].plot(kind='kde', ax=ax[ , ])
데이터['변수3(연속형)'][데이터['변수1(카테고리형)'] == 범주3].plot(kind='kde', ax=ax[ , ])
...
ax[ , ].xlabel('변수3(연속형)')
ax[ , ].title('변수3 Distribution within classes')
ax[ , ].legend('1st Class', '2nd Class', '3rd Class', ...)

ax[ , ].set_title('(3) 변수 vs 타겟')

## 2.3 이변수 - Target 분석
- 두 변수에 관하여 Target이 어떻게 다른지를 확인

### 2.3.1 두 변수 모두 카테고리형, 서수형 변수
- 변수1의 각 범주에서 변수2의 범주에 따라 Target이 어떻게 달라지는지 확인
- 변수2의 각 범주에서 변수1의 범주에 따라 Target이 어떻게 달라지는지 확인

In [ ]:
binary = meta[(meta.level == 'binary') & (meta.keep)].index

#### 꺾은선 그래프로 시각화

In [ ]:
# x축에 변수1, y축에 타겟으로 하고 변수2의 그래프를 추가
sns.factorplot('변수1', '타겟', hue또는col='변수2', data=데이터, size=5, aspect=1.5)

# x축에 변수2, y축에 타겟으로 하고 변수1의 그래프를 추가
sns.factorplot('변수2', '타겟', hue또는col='변수1', data=데이터, size=5, aspect=1.5)

# hue는 합쳐서, col은 나눠서, hue와 col을 동시에 사용할 수도 있음
# kind='bar'로 하면 그래프가 아니나 막대형태 표현 가능

# catplot or factorplot 은 figure 수준의 plot이므로 subplot을 듣지 않음
# factorplot은 없어짐, catplot 알아볼 것

#### crosstab으로 시각화

In [ ]:
pd.crosstab([데이터['변수1'], data['타겟']], data['변수2'], margins=True).style.background_gradient(cmap='summer_r')

#### pairplot으로 시각화

In [ ]:
sample = 데이터.sample(frac=0.05)
var = ['변수1', '변수2', '타겟']
sample = sample[var]
sns.pairplot(sample, hue='타겟', palette='Set1', diag_kind='kde')
plt.show()

### 2.3.2 변수1: 카테고리, 변수2: 연속형

#### 변수2(연속형) 값에 따른 Target kde를 변수1(카테고리형)의 각 범주마다 시각화

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(30, 8))

# 변수1(카테고리) 범주 1
sns.kdeplot(데이터[(데이터['타겟'] == 0) & (데이터['변수1(카테고리형)'] == 범주1)]['변수2(연속형)'], ax=ax[0])
sns.kdeplot(데이터[(데이터['타겟'] == 1) & (데이터['변수1(카테고리형)'] == 범주1)]['변수2(연속형)'], ax=ax[0])
ax[0].legend(['타겟 == 0', '타겟 == 1'])
ax[0].set_title('변수1의 범주1')

# 변수1(카테고리) 범주 2
sns.kdeplot(데이터[(데이터['타겟'] == 0) & (데이터['변수1(카테고리형)'] == 범주2)]['변수2(연속형)'], ax=ax[1])
sns.kdeplot(데이터[(데이터['타겟'] == 1) & (데이터['변수1(카테고리형)'] == 범주2)]['변수2(연속형)'], ax=ax[1])
ax[1].legend(['타겟 == 0', '타겟 == 1'])
ax[1].set_title('변수1의 범주2')

# 변수1(카테고리) 범주 3
sns.kdeplot(데이터[(데이터['타겟'] == 0) & (데이터['변수1(카테고리형)'] == 범주3)]['변수2(연속형)'], ax=ax[2])
sns.kdeplot(데이터[(데이터['타겟'] == 1) & (데이터['변수1(카테고리형)'] == 범주3)]['변수2(연속형)'], ax=ax[2])
ax[2].legend(['타겟 == 0', '타겟 == 1'])
ax[2].set_title('변수1의 범주3')

plt.show()

# 커널 함수식으로 추정한 그래프이며 정확한 히스토그램은
데이터['변수2(연속형)'][(데이터['변수1(카테고리형)'] == 1) & (데이터['타겟'] == 0)].plot(kind='hist')

- 변수2(연속형)에 따른 Target의 분포를 변수1의 각 범주마다 분리해서 시각화
- 이 방식이 violinplot보다 가독성이 좋음

#### 연속형 변수를 y축으로 놓고 violinplot 시각화를 하면

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 8))

sns.violinplot('변수1(카테고리형)', '변수2(연속형)', hue='타겟', data=데이터, scale='count', split=True, ax=ax)
ax.set_title('변수1 and 변수2 vs 타겟')
ax.set_yticks(range(최솟값, 최댓값, 간격))
plt.show()

# scale='area'는 면적을 같게 해서 distribution 차이를 보기 쉽게,
# scale='count'는 실제 관찰값의 개수를 알아보기 쉽게
# scale='width'는?

#### 연속형 변수를 카테고리화 한 후 factorplot으로도 시각화 가능(아직 categorize안 했으므로 plot으로 구현)
- 때로는 이 방법이 변수관계를 더욱 효과적으로 보여준다

In [ ]:
# 나눌 구간 확인
데이터['변수_range'] = pd.qcut(data['변수'], 구간개수) 
데이터.groupby(['변수_range'])['타겟'].mean().to_frame().style.background_gradient(cmap='summer_r')

# groupby후 mean()을 하면 Series가 되기때문에 style을 쓸수 없음 => to_frame()으로 DataFrame화

# qcut은 샘플수를 최대한 맞춰주면서 분할
# cut은 그냥 간격을 똑같이 맞춤(한 쪽 class로 데이터가 몰릴 수도 있다
# 둘 다 해보고 value_counts()로 믿을만 한지 확인해봐야 함

In [ ]:
plt.figure(figsize=(20, 6))

plt.subplots( , , 1)
변수1_범주1 = []
for i in range(최솟값, 최댓값, 간격):
    변수1_범주1.append(데이터[(데이터['변수2'] >= i) & (데이터['변수2'] < i + 간격) & (데이터['변수1'] == 범주1)]['타겟'].mean())
plt.plot(변수1_범주1)
plt.title('타겟 rate in 변수1 범주1')
plt.ylabel('타겟 rate')
plt.xlabel('변수2')
plt.xticks(np.arange(구간))
plt.ylim(0, 1)

plt.subplots( , , 2)
변수1_범주2 = []
for i in range(최솟값, 최댓값, 간격):
    변수1_범주1.append(데이터[(데이터['변수2'] >= i) & (데이터['변수2'] < i + 간격) & (데이터['변수1'] == 범주2)]['타겟'].mean())
plt.plot(변수1_범주2)
plt.title('타겟 rate in 변수1 범주2')
plt.xlabel('변수2')
plt.xticks(np.arange(구간))
plt.ylim(0, 1)

plt.subplots( , , 3)
변수1_범주3 = []
for i in range(최솟값, 최댓값, 간격):
    변수1_범주1.append(데이터[(데이터['변수2'] >= i) & (데이터['변수2'] < i + 간격) & (데이터['변수1'] == 범주3)]['타겟'].mean())
plt.plot(변수1_범주3)
plt.title('타겟 rate in 변수1 범주3')
plt.xlabel('변수2')
plt.xticks(np.arange(구간))
plt.ylim(0, 1)

    
# 간격을 정하는 것이 매우 중요
# y축 스케일 일치시켜야 한다

#### pairplot으로 시각화

In [ ]:
sample = 데이터.sample(frac=0.05)
var = ['변수1', '변수2', '타겟']
sample = sample[var]
sns.pairplot(sample, hue='타겟', palette='Set1', diag_kind='kde')
plt.show()

### 2.3.3 두 변수 모두 연속형인 경우

In [ ]:
v = meta[(meta.level == 'interval') & (meta.keep)].index

#### 회귀직선 그리기
- 상관관계가 나타나는 변수들을 직접 그려보기

In [ ]:
s = train.sample(frac=0.1)  # 데이터 개수가 너무 많으면 표본 일부만 추출해서 그리기

sns.lmplot(x='변수1', y='변수2', data=데이터, hue='타겟', palette='Set1', scatter_kws={'alpha':0.3})

- alpha를 0.3으로 했으므로 색이 진하면 값이 많아서 겹쳤다는 의미
- target값 0과 1의 회귀직선이 다르면 변수간 상호작용 효과가 있다는 의미

#### pairplot으로 시각화

In [ ]:
sample = 데이터.sample(frac=0.05)
var = ['변수1', '변수2', '타겟'] # 모든 연속 형 변수를 넣어서 그릴 수도 있음
sample = sample[var]
sns.pairplot(sample, hue='타겟', palette='Set1', diag_kind='kde')
plt.show()

- target이 1인 것과 0인것의 분포 차이가 나타나는 변수조합을 찾아낼 수 잇음

#### kde plot으로 시각화

In [ ]:
sns.kdeplot(trainset[trainset['target'] != 0]['ps_reg_01'], bw=, label='target = 1')
sns.kdeplot(trainset[trainset['target'] == 0]['ps_reg_01'], bw=, label='target = 0')
plt.xlim([, ])

- bw: 커널 크기를 지정하는 듯?

## 2.4 세 개의 변수 - Target 분석

### 2.4.1 변수가 모두 카테고리형 변수

#### crosstab

In [ ]:
pd.crosstab([데이터['변수1'], 데이터['변수2']], 
            [데이터['변수3'], 데이터['타겟']], margins=True).style.background_gradient(cmap='summer_r')

- 변수1, 변수2는 x축, 변수3, 타겟은 y축
- 타겟의 분포를 각각의 변수로 모두 split해서 살펴볼 수 있다

#### factorplot

In [ ]:
sns.factorplot('변수1', '타겟', hue='변수2', col='변수3', size=5, aspect=1.5)

# EDA시 주의사항

- 1) 연속형 변수를 구간화해서 x축으로 둔 시각화를 할 때 연속형 변수가 정규분포면 각 구간이 균등하지 않음 
      => 양쪽 끝 구간은 표본이 크지 않다 => 이상한 값이 나올 가능성이 크다
      - 이런 현상은 카테고리값에서 각 범주가 균등하지 않을때도 발생, 데이터가 적은 범주에서 도출된 결과는 신뢰성이 낮음
- 2) log변환 등 처리를 null값 처리 후에 해야 하므로 Feature Engineering단계에서 log변환 후 해당 변수에 대해 EDA단계로 돌아와야 함

#### 데이터에서 무언가 찾아낼 수 있는 것, 또는 사례
- porto-seguro-exploratory-analysis-and-prediction 노트북에서
    - 'ps_car_15', 'ps_car_15' 칼럼이 원래 값에서 10으로 나눈 후 제곱근을 씌운 값이라는 것을 알아냄
- 데이터 값이 0 ~ 1사이 값이면 어떤 비율을 의미하는 것일 수 있음

# 3. Feature Engineering
- **train, valid, test중 어디에 적용해야할 지 주의**

## 3.1 Null값 처리
- Null값이 존재하는 칼럼이 몇개인지 다시 확인
- 대부분의 Feature engineering 전에 해야 함 ex) log
- train데이터에서의 추측값으로 valid, test를 채워야 한다(valid는 나중에 valid 데이터로 채울수 있을듯)

### 3.1.1 사용하지 않음
- Null값이 너무 많은 칼럼은 정보가 별로 없으므로 나중에 drop()
- meta데이터에서 keep=False로 초기화

In [ ]:
# binary, nominal, ordinal 변수의 경우

데이터[['변수', '타겟']].groupby('변수').mean()

- NaN 값 자체가 정보가 될 수도 있으니 체크해 봐야함

In [ ]:
데이터.drop('변수', inplace=True, axis=1)
meta.loc[(vars_to_drop), 'keep'] = False # Updating the meta

### 3.1.2 단순 통계량으로 처리
- Null값 개수가 몇 개 안되거나 특정값이 매우 많은 경우 가장 많은 값으로 치환

#### 카테고리 변수

In [ ]:
# pandas 방법, 가장 많은 값으로 치환
훈련['변수'].fillna('가장많은 값', inplace=True)
테스트['변수'].fillna('가장많은 값', inplace=True)

# sklearn 이용한 방법
from sklearn.impute import SimpleImputer

어떤방법_imp = SimpleImputer(missing_values=nan, strategy='most_frequent')
훈련['변수'] = 어떤방법_imp.fit_transform(훈련[['변수']]).ravel()
테스트['변수'] = 어떤방법_imp.transform(테스트[['변수']]).ravel()

In [ ]:
# null값이 채워졌는지 확인
데이터['변수'].isnull().sum()  # 또는 any()

#### 연속형 변수

In [ ]:
# 중위값으로 채우는 경우
데이터['변수'] = 데이터['변수'].fillna(데이터['변수'].median())

# 평균으로 채우는 경우
데이터['변수'] = 데이터['변수'].fillna(데이터['변수'].mean())

### 3.1.3 다른 Feature의 정보로부터 추측할 수 있는 경우

### 3.1.3.1 채울 Feature: 연속형, 추측 Feature: 카테고리형

#### boxplot으로 Feature들과 분포 연관성 비교

In [ ]:
# boxplot
f, ax = plt.subplots(1, 2, figsize=(20, 12))

sns.boxplot(y='채울변수', x='변수1', data=데이터, ax=ax[0, 0])
sns.boxplot(y='채울변수', x='변수1', hue='변수2', data=데이터, ax=ax[0, 1])

# hue=변수2(카테고리형)을 추가하면 변수1를 또다른 변수2로 쪼갤 수 있음

# sns.factorplot(y='채울변수', x='변수1', col='변수2', data=데이터, kind='box') 변수2로 쪼갠 범주를 각 figure로 분산

- boxplot보다 kdeplot을 겹치는게 더 가독성이 좋음(이변수-target분석-연속형, 카테고리형 참고)

In [ ]:
# 두 개의 이상의 Feature를 이용한 mean 혹은 median 계산할 수도 있다
데이터.groupby(['추정변수1', '추정변수2'])['채울변수'].mean()

데이터.groupby(['추정변수1', '추정변수2'])['채울변수'].count() # 데이터 개수가 신뢰성 있을만큼 많은지 확인

- 두 개 이상의 Feature를 이용해 null값을 채울 수도 있다

#### 상관계수로 비교

In [ ]:
# 상관계수
sns.heatmap(dataset[['Age', 'Sex', 'SibSp', 'Parch', 'Pclass']].corr(), cmap='BrBG', annot=True)

# 카테고리형 변수의 string 값을 numerical 값으로 변환해야 함

#### 또 다른 방법들

- 타이타닉 튜토리얼 : 이름의 Mr., Mrs., Master, Ms., Miss, master, Dr. 등 에서 나이를 추측

#### Null 값 채우는 함수들

In [ ]:
# for문 방법, 다른 feature의 대푯값으로 채우고 그것도 NaN이면 채울변수 대푯값으로 채움

index_NaN_채울변수 = list(데이터['채울변수'][dataset['채울변수'].isnull()].index) # 채울변수 null값 인덱스 리스트

채울변수_med = 데이터['채울변수'].median() 혹은 mean()
for i in index_NaN_age:
    채울변수_pred = 데이터['채울변수'][((데이터['추정변수1'] == 데이터.iloc[i]['추정변수1']) & 
                                       (데이터['추정변수2'] == 데이터.iloc[i]['추정변수2']) & 
                                       (데이터['추정변수3'] == 데이터.iloc[i]['추정변수3']))].median() 혹은 mean()
    if not np.isnan(채울변수_pred):             # pandas는 .isnull()  numpy는 .isnan()
        데이터['채울변수'].iloc[i] = 채울변수_pred
    else:
        데이터['채울변수'].iloc[i] = 채울변수_med
        


# 하드코딩 방법
# train
데이터_train.loc[(데이터_train['채울변수'].isnull()) & (데이터_train['추정변수1'] == '범주1') & (데이터_train['추정변수2'] == '범주1'), '채울변수'] = 
데이터_train.loc[(데이터_train['채울변수'].isnull()) & (데이터_train['추정변수1'] == '범주1') & (데이터_train['추정변수2'] == '범주2'), '채울변수'] = 
데이터_train.loc[(데이터_train['채울변수'].isnull()) & (데이터_train['추정변수1'] == '범주1') & (데이터_train['추정변수2'] == '범주3'), '채울변수'] = 

데이터_train.loc[(데이터_train['채울변수'].isnull()) & (데이터_train['추정변수1'] == '범주2') & (데이터_train['추정변수2'] == '범주1'), '채울변수'] = 
데이터_train.loc[(데이터_train['채울변수'].isnull()) & (데이터_train['추정변수1'] == '범주2') & (데이터_train['추정변수2'] == '범주2'), '채울변수'] = 
데이터_train.loc[(데이터_train['채울변수'].isnull()) & (데이터_train['추정변수1'] == '범주2') & (데이터_train['추정변수2'] == '범주3'), '채울변수'] = 

데이터_train.loc[(데이터_train['채울변수'].isnull()) & (데이터_train['추정변수1'] == '범주3') & (데이터_train['추정변수2'] == '범주1'), '채울변수'] = 
데이터_train.loc[(데이터_train['채울변수'].isnull()) & (데이터_train['추정변수1'] == '범주3') & (데이터_train['추정변수2'] == '범주2'), '채울변수'] = 
데이터_train.loc[(데이터_train['채울변수'].isnull()) & (데이터_train['추정변수1'] == '범주3') & (데이터_train['추정변수2'] == '범주3'), '채울변수'] = 

# train에서의 통계값으로 test를 채움
valid혹은test.loc[(valid혹은test['채울변수'].isnull()) & (valid혹은test['추정변수1'] == '범주1') & (valid혹은test['추정변수2'] == '범주1'), '채울변수'] = 
valid혹은test.loc[(valid혹은test['채울변수'].isnull()) & (valid혹은test['추정변수1'] == '범주1') & (valid혹은test['추정변수2'] == '범주2'), '채울변수'] = 
valid혹은test.loc[(valid혹은test['채울변수'].isnull()) & (valid혹은test['추정변수1'] == '범주1') & (valid혹은test['추정변수2'] == '범주3'), '채울변수'] = 

valid혹은test.loc[(valid혹은test['채울변수'].isnull()) & (valid혹은test['추정변수1'] == '범주2') & (valid혹은test['추정변수2'] == '범주1'), '채울변수'] = 
valid혹은test.loc[(valid혹은test['채울변수'].isnull()) & (valid혹은test['추정변수1'] == '범주2') & (valid혹은test['추정변수2'] == '범주2'), '채울변수'] = 
valid혹은test.loc[(valid혹은test['채울변수'].isnull()) & (valid혹은test['추정변수1'] == '범주2') & (valid혹은test['추정변수2'] == '범주3'), '채울변수'] = 

데이터_valid혹은test.loc[(valid혹은test['채울변수'].isnull()) & (valid혹은test['추정변수1'] == '범주3') & (valid혹은test['추정변수2'] == '범주1'), '채울변수'] = 
데이터_valid혹은test.loc[(valid혹은test['채울변수'].isnull()) & (valid혹은test['추정변수1'] == '범주3') & (valid혹은test['추정변수2'] == '범주2'), '채울변수'] = 
데이터_valid혹은test.loc[(valid혹은test['채울변수'].isnull()) & (valid혹은test['추정변수1'] == '범주3') & (valid혹은test['추정변수2'] == '범주3'), '채울변수'] = 



# null값 채워졌는지 확인
데이터_train['Age'].isnull().sum()
데이터_test['Age'].isnull().sum()

## 3.2 log정규분포인 경우 log취할 수 있음

In [ ]:
# log 취하기전에 null값을 모두 채워야 한다. log취한 후 null값 채우면 스케일이 달라짐

데이터['변수'] = 데이터['변수'].map(lambda i: np.log(i) i > 0 else 0)
# train, valid, test에 모두 적용

# 정규분포가 되었는지 확인
# 실수로 log를 여러번 취할 수 있으므로 시각화 코드를 동시에 실행하여 실수 방지
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
sns.displot(데이터['변수'], color='b', label='Skewness : {:.2f}'.format(데이터['변수'].skew()), ax=ax)
ax.legend(loc='best')

## 3.3 Continuous 데이터의 Categorize
- 자칫 정보 손실이 일어날 수 있으니 주의해서 사용
- train에서 만든 범주로 valid, test에 모두 적용

#### 구간 개수에 따른 구간 범위

In [ ]:
데이터['변수_range'] = pd.qcut(data['변수'], 구간개수) # qcut는 샘플수를 최대한 일정하게 맞춰서 나눠줌
데이터.groupby(['변수_range'])['타겟'].mean().to_frame().style.background_gradient(cmap='summer_r')
# groupby후 mean()을 하면 Series가 되기때문에 style을 쓸수 없음 => to_frame()으로 DataFrame화

#### 하드코딩 방법

In [ ]:
데이터['변수_cat'] = 0
데이터.loc[데이터['변수'] <= 첫 구간, '변수'] = 0
데이터.loc[(데이터['변수'] <= 구간) & (데이터['변수'] > 구간), '변수_cat'] = 1
데이터.loc[(데이터['변수'] <= 구간) & (데이터['변수'] > 구간), '변수_cat'] = 2
데이터.loc[(데이터['변수'] <= 구간) & (데이터['변수'] > 구간), '변수_cat'] = 3
...
데이터.loc[(데이터['변수'] > 마지막 구간), '변수_cat'] = 9
# train, valid, test에 모두 적용

#### 함수를 이용한 apply 방법

In [ ]:
데이터['변수_cat'] = 0

def category_변수(x):
    if x < 10:
        return 0
    elif x < 20:
        return 1
    ...
    else:
        return 9
    
# train, valid, test에 모두 적용
데이터['변수_cat'] = 데이터['변수'].apply(category_변수)

#### pd.cut 방법

In [ ]:
데이터['변수_binned'] = pd.cut(데이터['변수'], bins=np.linspace(최소, 최대, 분할갯수))

#### sklearn.preprocessing KBinsDiscretizer 방법

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

kb = KBinsDiscretizer(n_bins=, strategy='uniform')
kb.fit(데이터)

X_binned = kb.transform(데이터)  # sparse-matrix를 반환
X_binned.toarray() # 원핫인코딩 된 array

#### 원래 변수 칼럼 삭제

In [ ]:
데이터.drop(['변수'], axis=1, inplace=True) # drop()함수는 axis=1 이 칼럼 drop이다

## 3.4 상호작용 변수
- 트리 모델의 경우 상호작용 변수를 자동으로 찾으므로 의미가 없거나 혹은 성능이 더 떨어질 수도 있다
- 뉴럴넷을 사용한다면? 은닉층이 상호작용을 자동으로 계산하는 역할을 할 듯?

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

poly.fit(훈련['변수1', '변수2'])
X_poly_train = poly.transform(훈련['변수1', '변수2'])
X_poly_test = poly.transform(테스트['변수1', '변수2'])
# transform은 numpy를 반환하므로 dataframe으로 만들어야 함
X_poly_train = pd.DataFrame(X_poly_train, columns=poly.get_feature_names(['변수1', '변수2']))
X_poly_test = pd.DataFrame(X_poly_test, columns=poly.get_feature_names(['변수1', '변수2']))

훈련데이터 = pd.concat([훈련, X_poly_train], axis=1)
테스트데이터 = pd.concat([테스트, X_poly_test], axis=1)

In [ ]:
# 각 feature의 이름을 얻는 속성
poly.get_feature_names(['넣었던 변수들 이름'])

- include_bias : 절편에 해당하는 1인 특성을 추가할 것인가

#### 상호작용 다른 예
- Home credit 대회 LightGBM 7th place solution 노트북
    - 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3' 변수에 min, max, mean, nanmedian, var등의 aggregation 변수 생성
    - do_mean, do_median등 여러 feature groupby한 feature 생성

## 3.5 Category Feature의 어떤 한 범주값을 binary화
- Home Credit competition(start here a gentle introduction)

In [ ]:
데이터['Is범주'] = 0
데이터.loc[데이터['변수(카테고리)'] == 범주, 'Is범주'] = 1

# 훈련, 테스트 데이터 모두 바꿔야 함

## 3.6 칼럼에서 정보를 추출하는 사례들

- Titanic Top 4% with ensemble modeling: Ticket 접두어 처리(string Feature 처리)

## 3.7 String => Numerical
- 값이 문자열인 카테고리 변수를 정수 인코딩
- 가능한 나중에 하는 것이 좋음. 미리 해놓으면 범주값 식별이 힘들듯?
- train, valid, test에 모두 적용

#### 변수 갯수 확인

In [ ]:
데이터['변수'].value_counts() # series
데이터['변수'].unique() # array

In [ ]:
# apply 방법
데이터['변수'] = 데이터['변수'].map({'범주0': 0, '범주1': 1, ... 'Other': 9})

# replace를 이용한 방법
데이터['변수'].replace(['범주1', '범주2', '범주3'], [0, 1, 2], inplace=True)

# pandas.factorize   (LabelEncoder보다 2배 가량 빠름)
데이터['변수'], uniques = pd.factorize(데이터['변수'])

# 한꺼번에 바꾸는 함수
def label_encoder(df, categorical_columns=None):
    """Encode categorical values as integers (0,1,2,3...) with pandas.factorize. """
    if not categorical_columns:
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    for col in categorical_columns:
        df[col], uniques = pd.factorize(df[col])
    return df, categorical_columns

#### sklearn.LabelEncoder 방법

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# 이진 데이터
binary = meta[(meta.level == 'binary') & (meta.keep)].index
binary_string = binary[['string변수1', 'string변수2']]

for b in binary_string:
    le = LabelEcoder()
    le.fit(훈련[b])
    훈련[b] = le.transform(훈련[b])
    테스트[b] = le.transform(테스트[b])

In [ ]:
# 카테고리 데이터
category = meta[(meta.level == 'categorical') & (meta.keep)].index
category_string = category[['string변수1', 'string변수2']]

for c in category_string:
    le = LabelEncoder()
    le.fit(훈련[c])
    훈련[c] = le.transform(훈련[c])
    테스트[c] = le.transform(테스트[c])

## 3.8 두 칼럼(변수)의 값을 합쳐야 하는 경우
- Sibling + Spouse + Parents + Child = Family

In [ ]:
데이터['새 변수'] = 데이터['변수1'] + 데이터['변수2']
# train, valid, test에 모두 적용

# 새로 만든 변수로 EDA를 한다
데이터['새변수'].value_counts()

## 3.9 변수간의 관계 확인(원-핫인코딩 전에 수행)
- string값을 정수인코딩 한 후에 해야 한다
- Target을 제외한 변수들 간에 -1 혹은 1인 값들이 존재할 경우 redundant데이터, 필요없다
- 다른 변수에서의 정보로 어떤 변수의 null값을 채울 경우 상관관계가 나타날 수 있음

In [ ]:
heatmap_data = 데이터[['변수1', '변수2', ...]]

colormap = plt.cm.RdYlGn
plt.figure(figsize=(14, 8))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), 
            linewidths=0.1, vmax=1.0, cmap=colormap, linecolor='white', annot=True, annot_kws={'size': 16}, fmt='.2f')

# data.corr() --> correlation matrix
# linewidths: 칸 간격, vmax: 색깔 최댓값, annot: 수치표시, fmt: 소수점

- 히트맵이 까맣게 나오는 이유
    - 변수가 아주 많은 경우에 annot=True로 주석 글씨 때문
- 히트맵이 모두 하얗게 나오는 이유
    - 변수가 아주 많은 경우에 linecolor='white', linewidth 때문

#### 변수가 너무 많아서 heatmap으로 못 그릴 때는 수치로 확인

In [ ]:
# Find correlations with the target and sort
correlations_target = 데이터.corr()['타겟'].sort_values()
correlations = 데이터.corr()

# Display correlations_target
print('Most Positive Correlations:\n', correlations_target.tail(15))
print('\nMost Negative Correlations:\n', correlations_target.head(15))

## 3.10 Categorical 변수 인코딩


### 3.10.1 One-hot Encoding
- 카테고리(nominal) 데이터만 원핫인코딩, 순서형(Ordinal) 변수는 One-Hot Encoding 하지 않음
- from sklearn.preprocessing import OneHotEncoder
- 선형 모델에서 사용, 트리 모델에서는 유용하지 않음
- high cardinality로 인해 너무 많은 feature가 생길 경우 tree-base model에서 samples a fraction of features로 split시 not one-hot encoded feature에 기회가 적게 돌아간다

In [ ]:
데이터["변수"] = 데이터['변수'].astype('category')
데이터 = pd.get_dummies(데이터, columns=['변수'], prefix='원핫인코딩의 접두사', dummy_na=True)

# 카테고리 변수들을 한꺼번에 처리(meta는 train으로 만들었으므로 train만 적용되었음, test도 수행)
v = meta[(meta.level == 'nominal') & (meta.keep)].index
훈련데이터 = pd.get_dummies(훈련데이터, columns=v, drop_first=)

# pd.get_dummies()로 원핫인코딩을 할 경우 훈련데이터의 카테고리 변수에는 있는 범주값이 테스트 데이터에터에는 없어서
# 테스트 데이터의 해당 범주값의 원핫인코딩 feature가 안생길 수도 있으므로 그런 범주를 삭제
# 약간의 정보손실이 생기는 것이므로 주의. 
# 이렇게 하지 않으려면 scikit-learn의 one-hot encoder 사용
훈련_타겟 = 훈련['타겟'] 
훈련데이터, 테스트데이터 = 훈련데이터.align(테스트데이터, join='inner', axis=1)




- train, valid, test에 모두 적용
- get_dummies함수는 원래 칼럼을 삭제한다, 문자열 칼럼에 적용됨, integer 칼럼에는 적용 안됨
    - dummy_na: null값에 대한 feature를 만들지 결정
- train, valid, test의 원핫인코딩이 같은 값으로 인코딩 되었는지 확인

### 3.10.2 Mean Encoding

- 카테고리 변수의 범주 개수가 너무 많은 경우
- 각 범주값들을 해당 범주에서의 타겟 값의 비율로 치환하고 이 변수를 연속형 변수 취급할 수 있음
- 오버피팅 가능성이 있음(각 범주에서 데이터 개수가 충분해야 함)
    - 베르누이 기댓값은 np > 5 , n(1 - p) > 5 가 만족되어야 정규분포를 따름
    - 대부분의 high cardinality feature들은 각 범주에서 sample이 별로 없다는게 문제
    - 베르누이 기댓값이 0.5, 때 n이 10인 경우 95% 신뢰구간은 +-0.3

#### 첫 번째 방법

In [ ]:
# porto competitions : xgboost-cv-lb-284
# 오버피팅을 막기 위해 노이즈를 넣어줌
def add_noise(Series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None,
                  val_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following by Daniele Micci-Barreca
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(['mean', 'count'])
    
    # Compute Smoothing(use sigmoid function)
    smoothing = 1 / (1 + np.exp(-(averages['count'] - min_samples_leaf) / smoothing))
    # 각 범주의 관측된 값이 많을수록 smoothing 값이 1에 가까워짐, 적으면 smoothing값이 0에 가까워짐
    
    # Apply average function to all target data
    prior = target.mean()
    
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages['mean'] * smoothing
    averages.drop(['mean', 'count'], axis=1, inplace=True)
    
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}), 
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so resotre it
    ft_tst_series.index = tst_series.index
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [ ]:
# train, test 모두 바꿔줘야 한다

train_encoded, val_encoded, test_encoded = target_encode(훈련['변수'], 
                                                         검증['변수'], 
                                                         테스트['변수'], 
                                                         min_samples_leaf=100, 
                                                         smoothing=10, 
                                                         noise_level=0.01)

훈련['변수_te'] = train_encoded
훈련.drop('변수', axis=1, inplace=True)
meta.loc['변수', 'keep'] = False   # Updating the meta

검증['변수_te'] = val_encoded

테스트['변수'] = test_encoded
테스트.drop('변수', axis=1, inplace=True)

각 범주값마다 target(0, 1)의 가중평균값을 계산하겠다는 것인데

관찰값이 많은 범주의 경우 그 범주에서의 target의 mean값에 높은 가중치를 주고

관찰값이 적은 범주의 경우 target의 전체 mean값에 높은 가중치를 줘서 가중 평균한다

### Regularized mean encoding
- high cardinality인 경우 4 ~ 5 fold, alpha 5를 적용하는게 좋음
- mean encoding과 frequency encoding 둘다 쓰는게 좋을 수도 있다
- 데이터와 cardinality가 매우 클 경우 expanding도 고려 가능
- low cardinality에서는 label or frequency 인코딩을 추천

In [ ]:
def mean_encode(train_data, test_data, columns, target_col, reg_method=None,
                alpha=0, add_random=False, rmean=0, rstd=0.1, folds=1):
    '''Returns a DataFrame with encoded columns'''
    encoded_cols = []
    target_mean_global = train_data[target_col].mean()
    for col in columns:
        # Getting means for test data
        nrows_cat = train_data.groupby(col)[target_col].count()     # 각 범주에서 샘플 갯수
        target_means_cats = train_data.groupby(col)[target_col].mean()    # 각 범주에서 target_mean
        target_means_cats_adj = (target_means_cats*nrows_cat + target_mean_global*alpha) / (nrows_cat+alpha)  # 가중 평균
        
        # Mapping means to test data
        encoded_col_test = test_data[col].map(target_means_cats_adj)    # map 함수에 Series를 넣을 수 있음
        
        # Getting a train encodings
        if reg_method == 'expanding_mean':  # 각 범주값 내에서 무작위를 또 추가할 수 있음
            train_data_shuffled = train_data.sample(frac=1, random_state=1)
            cumsum = train_data_shuffled.groupby(col)[target_col].cumsum() - train_data_shuffled[target_col] # train_data_shuffled.groupby(col)[target_col].cumsum()  각 범주에서 target 누적합을 의미
            cumcnt = train_data_shuffled.groupby(col).cumcount() # 각 범주값이 몇번째 중복되는지를 계산 0이면 해당 범주값이 처음 나왔음을 의미
            encoded_col_train = cumsum / (cumcnt) # 각 범주에서 target mean값을 누적 계산
            encoded_col_train.fillna(target_mean_global, inplace=True)   # 한번도 안 나온 범주값은 target 전체 평균로 대체
            if add_random:
                encoded_col_train = encoded_col_train + normal(loc=rmean, scale=rstd, size=(encoded_col_train.shape[0]))  # 노이즈 추가
        
        elif (reg_method == 'k_fold') and (folds > 1):
            kfold = StratifiedKFold(train_data[target_col].values, folds, shuffle=True, random_state=1)
            parts = []
            for tr_in, val_ind in kfold:
                # divide data
                df_for_estimation, df_estimated = train_data.iloc[tr_in], train_data.iloc[val_ind]
                
                # getting means on data for estimation (all folds except estimated)
                nrows_cat = df_for_estimation.groupby(col)[target_col].count()
                target_means_cats = df_for_estimation.groupby(col)[target_col].mean()
                target_means_cats_adj = (target_means_cats*nrows_cat + target_mean_global*alpha) / (nrows_cat+alpha)
                
                # Mapping means to estimated fold
                encoded_col_train_part = df_estimated[col].map(target_means_cats_adj)
                if add_random:
                    encoded_col_train_part = encoded_col_train_part + normal(loc=rmean, scale=rstd, size=(encoded_col_train_part.shape[0]))
                
                # Saving estimated encodings for a fold
                parts.append(encoded_col_train_part)
            encoded_col_train = pd.concat(parts, axis=0)
            encoded_col_train.fillna(target_mean_global, inplace=True)
            
        else:
            encoded_col_train = train_data[col].map(target_means_cats_adj)
            if add_random:
                encoded_col_train = encoded_col_train + normal(loc=rmean, scale=rstd, size=(encoded_col_train.shape[0]))
    
    return incoded_col_train, encoded_col_test



In [ ]:
train_encoded, test_encoded = mean_encode(훈련데이터, 테스트데이터, 카테고리칼럼리스트, 훈련['타겟'], reg_method=None, 
                                          alpha=0, add_random=False, rmean=0, rstd=0.1, folds=1)


- test데이터에는 노이즈 추가나 regularization 하지 않음

### 3.10.3 Frequency encoding

#### 한 번에 하나씩
- 기존 칼럼을 남겨 놓음
- 테스트 데이터 처리 안했음

In [ ]:
def frequency_encoding(train_data, test_data, col):
    freq_encoding = train_data.groupby([col]).size() / train_data.shape[0]   # size includes NaN values, count does not
    freq_encoding = freq_encoding.reset_index().rename(columns={0:'{}_Frequency'.format(col)})
    
    return 데이터.merge(freq_encoding, on=col, how='left')

#### 다른방법(카테고리 칼럼들을 넣어주면 한번에 계산)
- 기존 칼럼을 대체
- 테스트 데이터도 처리

In [ ]:
def freq_encode(train_data, test_data, columns):
    '''Returns a DataFrame with encoded columns'''
    
    for col in columns:
        freqs_cat = train_data.groupby(col)[col].count() / train_data.shape[0]
        encoded_col_train = train_data[col].map(freqs_cat)
        encoded_col_test = test_data[col].map(freqs_cat)
        encoded_col = pd.concat([encoded_col_train, encoded_col_test], axis=0)
        encoded_col[encoded_col.isnull()] = 0
    
    return encoded_col_train, encoded_col_test

### 3.10.4 기타 카테고리 변수 인코딩 응용
- porto competition 2등 솔루션: 'ind'변수들을 여러개 묶어서 frequency encoding, 
    - high cardnality 변수가 많을 때 유용
    - 거기에 원핫인코딩도 같이 사용해서 변수를 표현
- Home Credit 대회 LightGBM 7th place solution노트북
    - 여러 변수들 묶어서 do_mean, do_median등으로 인코딩

## 3.11 특성 선택

### 3.11.1 분산이 낮은 변수 삭제
- 해놓으면 학습속도 높일 수 있음

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold(threshold=0.01)
selector.fit(데이터.drop(['단순ID', '타겟'], axis=1))  # Fit to train without id and target variables

f = np.vectorize(lambda x : not x)   # Function to toggle boolean array elements
v = train.drop(['단순ID', '타겟'], axis=1).columns[f(selector.get_support())] # ~numpy.array(list) 하는 방법도 있다

print('{} variables have too low variance.'.format(len(v)))
print('These variables are {}'.format(list(v)))

- valid, test에서도 삭제해야 되나?

### 3.11.2 일변량 통계

### 3.11.3 모델 기반 특성 선택
- 트리 기반 모델의 feature_importances_
- 선형모델의 계수 절댓값, L1 규제 등

#### RandomForest
- 일반적으로 일변량 분석보다 훨씬 강력

In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train = 데이터.drop(['단순아이디', '타겟'], axis=1)
y_train = 데이터['target']

feat_labels = X_train.columns

rf = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
rf.fit(X_train, y_train)

importances = rf.feature_importances_

for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f+1, 30, feat_labels[indices[f]], importances[f]))

In [ ]:
from sklearn.feature_selection import SelectFromModel

sfm = SelectFromModel(rf, threshold='median', prefit=True)
print('Number of features before selection: {}'.format(X_train.shape[1]))

n_features = sfm.transform(X_train).shape[1]
print('Number of features after selection: {}'.format(n_features))

selected_vars = list(feat_labels[sfm.get_support()])
데이터 = 데이터[selected_vars + ['target']]

- prefit: False로 하면 여기서 새로 다시 훈련

### 3.11.4 반복적 특성 선택(Recursive feature elimination)
- 10개씩 또는 20개씩
- 20 모델 baseline
- random choosing 20
- 40 모델 학습 -> baseline 비교
- if 성능 향상 -> feature importance 상위 10%에 새로 추가된 거 생기면
- 걔를 남기고
- 안생기면
- 다른 거 randomly choosing 
- 반복

### 3.11.5 전문가 지식 활용 사례

- Home Credit Competition의 start here a gentle introduction노트북
    - CREDIT_INCOME_PERCENT, ANNUITY_INCOME_PERCENT, CREDIT_TERM, DAYS_EMPLOYED_PERCENT 4개의 새로운 feature를 만들어 냄
    - LIGHTGBM에서 효과 있었음

#### 주의사항
- 도메인 지식으로 새로운 feature를 만들었다면 이 feature에 대해서도 EDA를 진행할 것
- 그리고 그 새로운 feature가 중요한지 확인하려면 feature_importances_

#### 필요 없는 칼럼 drop

In [ ]:

# train, valid, test 모두 적용
데이터.drop(['변수1', '변수2', ...], axis=1, inplace=True)

# df.drop은 특이하게 axis=1이 칼럼 삭제를 의미

### 그외 칼럼 선택, 삭제 사례

- 어떤 특정 문자로 시작하는 칼럼 삭제하려면 : porto-seguro-exploratory-analysis-and-prediction

## 3.12 스케일링

### 3.12.1 Standard scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
scaler.fit_transform(데이터.drop['타겟'], axis=1)

scaler = MinMaxScaler(feature_range=( , ))

# Pairplot 확인

In [ ]:
sns.pairplot(데이터[['변수1', '변수2', '변수3', '변수4', '변수5']], 
             hue='Survived', palette='seismic', size=1.2, diag_kind=dict(shade=True), plot_kws=dict(s=10))

# 메모리 사용량

In [ ]:
def reduce_memory(df):
    """Reduce memory usage of a dataframe by setting data types. """
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Initial df memory usage is {:.2f} MB for {} columns'
          .format(start_mem, len(df.columns)))

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            cmin = df[col].min()
            cmax = df[col].max()
            if str(col_type)[:3] == 'int':
                # Can use unsigned int here too
                if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif cmin > np.iinfo(np.int64).min and cmax < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    memory_reduction = 100 * (start_mem - end_mem) / start_mem
    print('Final memory usage is: {:.2f} MB - decreased by {:.1f}%'.format(end_mem, memory_reduction))
    return df

데이터 = reduce_memory(데이터)